# Dayfile read

In [1]:
import os
import struct
import sys

import pandas as pd

sys.path.append('../../')
from JohnsonUtil import commonTips as cct
from JSONData import tdx_data_Day as tdd



In [136]:
def readTdxLdayFile(fname="D:\\MacTools\\WinTools\\new_tdx\\vipdoc\\sh\\lday\\sh601628.day"):
    dataSet = []
    with open(fname, 'rb') as fl:
        buffer = fl.read()  # 读取数据到缓存
        size = len(buffer)
        rowSize = 32  # 通信达day数据，每32个字节一组数据
        code = os.path.basename(fname).replace('.day', '')
        for i in range(0, size, rowSize):  # 步长为32遍历buffer
            row = list(struct.unpack('IIIIIfII', buffer[i:i + rowSize]))
            row[0] = (cct.day8_to_day10(row[0]))
            row[1] = row[1] / 100
            row[2] = row[2] / 100
            row[3] = row[3] / 100
            row[4] = row[4] / 100
            row.pop()  # 移除最后无意义字段
            row.insert(0, code)
            dataSet.append(row)
    data = pd.DataFrame(data=dataSet, columns=[
                        'code', 'tradeDate', 'open', 'high', 'low', 'close', 'amount', 'vol'])
    print(data[-2:])



          code   tradeDate   open   high    low  close       amount       vol
1800  sh601628  2023-02-02  33.88  34.08  33.35  33.89  686049600.0  20307045
1801  sh601628  2023-02-03  33.85  33.85  33.15  33.51  526803712.0  15771846


In [164]:
readTdxLdayFile()

          code   tradeDate   open   high    low  close       amount       vol
1800  sh601628  2023-02-02  33.88  34.08  33.35  33.89  686049600.0  20307045
1801  sh601628  2023-02-03  33.85  33.85  33.15  33.51  526803712.0  15771846


In [124]:
df= tdd.get_tdx_Exp_day_to_df_AllRead_('601628', dl=800, resample='d').sort_index(ascending=True)

print(df[-1:],df.shape)

              code   open   high    low  close         vol       amount  \
date                                                                      
2023-02-03  601628  33.85  33.85  33.15  33.51  15771846.0  526803712.0   

             ma5d  ma10d  ma20d  ...  top10  ma5dcum  ra   hmax   max5   lmin  \
date                             ...                                            
2023-02-03  34.04   35.2  36.22  ...      0     35.8  10  50.67  37.43  33.35   

             min5  cmean          hv         lv  
date                                             
2023-02-03  34.61  34.17  25729948.0  8805922.0  

[1 rows x 111 columns] (800, 111)


In [109]:
df= tdd.get_tdx_Exp_day_to_df('601628', dl=80, resample='d').sort_index(ascending=True)
print(df[-1:],df.shape)

              code   open   high    low  close         vol       amount  \
date                                                                      
2023-02-03  601628  33.85  33.85  33.15  33.51  15771846.0  526803712.0   

             ma5d  ma10d  ma20d  ...  top10  ma5dcum  ra   hmax   max5   lmin  \
date                             ...                                            
2023-02-03  34.04   35.2  36.22  ...      0     35.8  10  38.05  37.43  33.35   

             min5  cmean          hv         lv  
date                                             
2023-02-03  34.61  34.17  25729948.0  8805922.0  

[1 rows x 111 columns] (79, 111)


# tdx 2
https://gist.github.com/WMJi/f5fe3083961d26fbfbcc
https://www.cnblogs.com/yuyanc/p/16867362.html

In [34]:
from struct import *

fname = "D:\\MacTools\\WinTools\\new_tdx\\vipdoc\\sh\\lday\\sh601628.day"
ofile = open(fname, 'rb')
buf = ofile.read()
ofile.close()

# ifile=open('sz000680.txt','w')
num = len(buf)
no = num / 32
b = 0
e = 32
line = ''
dataSet = []
# row=[]
for i in range(int(no)):
    code = os.path.basename(fname).replace('.day', '')
    a = list(unpack('IIIIIfII', buf[b:e]))
    # line=str(a[0])+' '+str(a[1]/100.0)+' '+str(a[2]/100.0)+' '+str(a[3]/100.0)+' '+str(a[4]/100.0)+' '+str(a[5]/10.0)+' '+str(a[6])+' '+str(a[7])+' '+'\n'
    # print(line)
    row = a
    row[0] = (a[0])
    row[1] = (a[1] / 100.0)
    row[2] = (a[2] / 100.0)
    row[3] = (a[3] / 100.0)
    row[4] = (a[4] / 100.0)
    row.pop()
    row.insert(0, code)
    # ifile.write(line)
    b = b + 32
    e = e + 32
    dataSet.append(row)
data = pd.DataFrame(data=dataSet, columns=[
                    'code', 'tradeDate', 'open', 'high', 'low', 'close', 'amount', 'vol'])
print(data)

          code  tradeDate   open   high    low  close        amount        vol
0     sh601628   20150902  24.60  26.44  24.50  26.43  2.650394e+09  102231049
1     sh601628   20150907  25.90  26.28  24.51  24.53  8.093017e+08   31744690
2     sh601628   20150908  24.30  25.58  24.10  25.46  6.499078e+08   25788785
3     sh601628   20150909  25.48  25.86  24.98  25.58  7.447775e+08   29322307
4     sh601628   20150910  25.29  25.48  25.04  25.40  4.120328e+08   16288522
...        ...        ...    ...    ...    ...    ...           ...        ...
1797  sh601628   20230130  36.84  36.87  34.61  34.80  8.439583e+08   24022646
1798  sh601628   20230131  34.86  35.23  33.96  34.04  6.805075e+08   19794678
1799  sh601628   20230201  34.12  34.26  33.35  33.94  8.301546e+08   24688664
1800  sh601628   20230202  33.88  34.08  33.35  33.89  6.860496e+08   20307045
1801  sh601628   20230203  33.85  33.85  33.15  33.51  5.268037e+08   15771846

[1802 rows x 8 columns]
